In [ ]:
%load_ext nb_black
import os
from pathlib import Path

# from tensorflow.keras.applications import (
#     MobileNetV2,
# )
from efficientnet.tfkeras import EfficientNetB0

In [ ]:
# dataset_name = "santander-customer-satisfaction"
# IMAGE_SIZE = 640
# dataset_name = "springleaf-marketing-response"
# IMAGE_SIZE = 1408
# dataset_name = "segment"
# IMAGE_SIZE = 160
# dataset_name = "rl"
# IMAGE_SIZE = 160
# dataset_name = "open-payments"
# target = "status"
# IMAGE_SIZE = 96
# dataset_name = "bank-marketing"
# IMAGE_SIZE = 160
# dataset_name = "springleaf-marketing-response"
# IMAGE_SIZE = 1408
# dataset_name = "bnp-cardif"
# IMAGE_SIZE = 384
# dataset_name = "albert"
# IMAGE_SIZE = 288
# dataset_name = "titanic"
# IMAGE_SIZE = 128
# target = "Survived"

# IMAGE_SIZE = 24
# dataset_name = "rl"
# target = "target"
# dataset_name = "bank-marketing"
# target = "y"
dataset_name = "give-me-some-credit"
target = "SeriousDlqin2yrs"

# dataset_name = "census-income"
# target = "taxable income amount"

# dataset_name = "homesite-quote-conversion"
# target = "QuoteConversion_Flag"

# dataset_name = "springleaf-marketing-response"
# target = "target
# dataset_name = "cat-in-the-dat-ii"
# IMAGE_SIZE = 96
# dataset_name = "give-me-some-credit"
# IMAGE_SIZE = 128

# IMAGE_SIZE = 160

DATASET_FOLDER = Path(os.getcwd()) / f"data/{dataset_name}"

BATCH_SIZE = 32
PREFETCH = 50


ONE_CHANNEL = True
NB_CHANNEL = 1 if ONE_CHANNEL else 3

# Target size for model to use
# TARGET_SIZE = IMAGE_SIZE
PRETRAINED_MODEL = EfficientNetB0  # MobileNetV2  # EfficientNetB0  # EfficientNetB4  # MobileNetV2  # EfficientNetB2  # EfficientNetB0  # MobileNetV2  # NASNetMobile

# Training params for only new layers

# Training params for refit
epochs = 200
patience = 5
BATCH_SIZE = 32
PREFETCH = 50

# Model name => image size, last_block retrain
# NASNetMobile => 224,
# NASNetLarge => 331,
# MobileNetV2 => 96, ... 160..... 224, last layer => 128 (3 blocks), 137(2 blocks), 146(1 block)
# InceptionV3 => 299,
# Xception => 299,

## Import + utilities

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.metrics import roc_auc_score, accuracy_score

from thc_net.image.pretrained_model import (
    build_dataset,
    build_process_path,
    build_model,
)

from thc_net.image.tabular_preproc import csv_to_pixel

%matplotlib inline

In [ ]:
def plot_metric(history, metric):
    # Plot training & validation loss values
    plt.plot(history.history[metric])
    plt.plot(history.history[f"val_{metric}"])
    plt.title(f"Model {metric}")
    plt.ylabel(f"{metric}")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Test"], loc="upper left")
    plt.show()

In [ ]:
!ls /work/data/census-income

## Preparing datasets

In [ ]:
import pandas as pd

panda_kwargs = {}
DATASET_FILENAME = "train_bench.csv"
dataset_path = DATASET_FOLDER / DATASET_FILENAME
TAILORED_COLUMN = "Set"

columns = pd.read_csv(dataset_path, **panda_kwargs, nrows=1).columns.tolist()
columns

In [ ]:
split = None
panda_kwargs = {}
if TAILORED_COLUMN not in columns:
    split = np.random.choice(
        ["train", "valid", "test"], p=[0.8, 0.1, 0.1], size=(NB_LINES,)
    )
else:
    split = pd.read_csv(
        dataset_path, **panda_kwargs, usecols=[TAILORED_COLUMN]
    ).values.reshape(-1)


train_indices = np.argwhere(split == "train").reshape(-1)
np.random.shuffle(train_indices)
valid_indices = np.argwhere(split == "valid").reshape(-1)
test_indices = np.argwhere(split == "test").reshape(-1)

In [ ]:
used_columns = list(set(columns) - set([TAILORED_COLUMN, target]))
used_columns

In [ ]:
df = pd.read_csv(dataset_path, low_memory=False)

In [ ]:
X = df[used_columns].values
Y = df[target].values.reshape(-1)

extract_df = df[used_columns]
nb_idx = (extract_df.dtypes == "int64") | (extract_df.dtypes == "float64")
nb_idx = extract_df.columns[nb_idx]
is_numeric = np.isin(extract_df.columns.values, nb_idx.values)

In [ ]:
(extract_df.nunique() / extract_df.shape[0]) < 0.02

In [ ]:
# https://www.tensorflow.org/tutorials/load_data/images
X, X_num, Y = csv_to_pixel(df, used_columns, target, ascii_only=ONE_CHANNEL)
IMAGE_SIZE = X.shape[1]
TARGET_SIZE = IMAGE_SIZE  # if IMAGE_SIZE > 32 else 32

In [ ]:
X.shape

In [ ]:
X_num.shape

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

Y = LabelEncoder().fit_transform(Y)

In [ ]:
Y = to_categorical(Y.reshape(-1, 1))
Y

In [ ]:
OUTPUT_DIM = len(np.unique(Y))
OUTPUT_DIM

In [ ]:
X.nbytes / 1024 ** 2

In [ ]:
from tensorflow.data import Dataset

In [ ]:
epochs = 2000
patience = 5
BATCH_SIZE = 1024
PREFETCH = 50

In [ ]:
dataset_train = (
    Dataset.from_tensor_slices(
        ((X[train_indices], X_num[train_indices]), Y[train_indices])
    )
    #     .repeat()
    .batch(BATCH_SIZE)
)
dataset_valid = (
    Dataset.from_tensor_slices(
        ((X[valid_indices], X_num[valid_indices]), Y[valid_indices])
    )
    #     .repeat()
    .batch(BATCH_SIZE)
)
dataset_test = Dataset.from_tensor_slices(
    ((X[test_indices], X_num[test_indices]), Y[test_indices])
).batch(BATCH_SIZE)

In [ ]:
for (image, num), label in dataset_train.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Num shape: ", num.numpy().shape)
    print("Label: ", label.numpy().shape)

### Images Example

In [ ]:
def show_image(image, image_size, one_channel=False):
    if one_channel:
        imshow(image.reshape(IMAGE_SIZE, IMAGE_SIZE))
    else:
        imshow(image)

In [ ]:
for (image, num), label in dataset_train.take(1):
    print("Label: ", label[0].numpy())
    print("Label: ", label[0].numpy())
    show_image(image[0].numpy(), IMAGE_SIZE, ONE_CHANNEL)

In [ ]:
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    Conv1D,
    Flatten,
    MaxPool2D,
    Dropout,
    BatchNormalization,
    Input,
    SpatialDropout2D,
    Concatenate,
    Reshape,
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.optimizers import Lookahead, RectifiedAdam
from tensorflow_addons.activations import mish, gelu
from tensorflow.keras.activations import swish

optimizer = Lookahead(RectifiedAdam(), sync_period=6, slow_step_size=0.5)
activation = "relu"

In [ ]:
inputs = Input(shape = (IMAGE_SIZE, IMAGE_SIZE, NB_CHANNEL))

inputs_num = Input(shape = (X_num.shape[1]))
x2 = Dense(X_num.shape[1]*5, activation = activation, kernel_initializer="he_normal")(inputs_num)
x2 = Dropout(rate=0.5)(x2)
x2 = Dense(X_num.shape[1]*2, activation = activation, kernel_initializer="he_normal")(x2)
x2 = Dropout(rate=0.2)(x2)
x2 = Dense(OUTPUT_DIM, activation=activation, kernel_initializer="he_normal")(x2)

#w2 = BatchNormalization()(inputs_num)

x=inputs
x = Conv2D(            
    kernel_size= (1, 1),
    padding= "SAME",
    activation= "relu",
    filters=32
)(inputs)
# x = SpatialDropout2D(0.5)(x)

# x = BatchNormalization()(x)

# x = Conv2D(            
#     kernel_size= (8, 8),
#     strides=(8,8),
#     padding= "SAME",
#     activation=activation,
#     kernel_initializer="he_normal",
#     filters=32
# )(x)
# x = Conv2D(            
#     kernel_size= (4, 4),
#     strides=(4, 4),
#     padding= "SAME",
#     activation=activation,
#     kernel_initializer="he_normal",
#     filters=64
# )(x)

# x = Conv2D(            
#     kernel_size= (8,8),
#     strides=(1, 1),
#     padding= "SAME",
#     activation=activation,
#     kernel_initializer="he_normal",
#     filters=128
# )(x)
# x = SpatialDropout2D(0.5)(x)

x = Conv2D(            
    kernel_size= (8,8),
    strides=(8,8),
    padding= "SAME",
    activation=activation,
#     kernel_initializer="he_normal",
    filters=16
)(x)
x = SpatialDropout2D(0.2)(x)

# x = Conv2D(            
#     kernel_size= (IMAGE_SIZE//8, IMAGE_SIZE//8),
#     strides=(1,1),
#     padding= "SAME",
#     activation=activation,
# #     kernel_initializer="he_normal",
#     filters=32
# )(x)

# x = Reshape(((IMAGE_SIZE//8)*(IMAGE_SIZE//8), 16))(x)
# x = Conv1D(            
#     kernel_size=16,
#     strides=1,
#     padding= "SAME",
#     activation=activation,
# #     kernel_initializer="he_normal",
#     filters=16
# )(x)

# x = MaxPool2D(pool_size=(4,4))(x)
# x = SpatialDropout2D(0.3)(x)


# x = Conv2D(            
#     kernel_size= (4, 4),
#     strides=(1, 1),
#     padding= "SAME",
#     activation= "relu",
#     filters=128
# )(x)

# x = Conv2D(            
#     kernel_size= (4, 4),
#     strides=(4, 4),
#     padding= "SAME",
#     activation= "relu",
#     filters=256
# )(x)

# x = BatchNormalization()(x)
# x = Conv2D(            
#     kernel_size= (1, 1),
#     padding= "SAME",
#     activation= "relu",
#     filters=128
# )(x)


# x = Conv2D(            
#     kernel_size= (1, 1),
#     padding= "SAME",
#     activation= "relu",
#     filters=32
# )(x)

# x = Conv2D(            
#     kernel_size= (1, 1),
#     padding= "SAME",
#     activation= "relu",
#     filters=16
# )(x)

# x = MaxPool2D(pool_size=(4,4))(x)
x = Flatten()(x)
x = Dense(OUTPUT_DIM, activation=activation, kernel_initializer="he_normal")(x)
# x = BatchNormalization()(x)

# x = BatchNormalization()(x)
# x2 = BatchNormalization()(x2)
x = Concatenate()([x, x2])

x = Dense(16, activation=activation, kernel_initializer="he_normal")(x)
# x = Dense((X_num.shape[1]*5+IMAGE_SIZE//8*8)*2, activation = activation, kernel_initializer="he_normal")(x)

# # x = Dropout(rate=0.2)(x)
# x = Dense((X_num.shape[1]*5+IMAGE_SIZE//8*8), activation = activation, kernel_initializer="he_normal")(x)
# # x = Dropout(rate=0.2)(x)

# if (X_num.shape[1]*5+IMAGE_SIZE//8*8) > 512:
#     x = Dense(128, activation = activation, kernel_initializer="he_normal")(x)
#     x = Dropout(rate=0.2)(x)

# x = Dense(512, activation = "relu")(x)
# x = Dropout(rate=0.2)(x)

# x = Dense(128, activation = "relu")(x)
# x = Dropout(rate=0.2)(x)

x = Dense(OUTPUT_DIM, activation = 'sigmoid')(x)

model = Model(inputs=[inputs, inputs_num], outputs=x)


In [ ]:
model.summary()

In [ ]:
#!pip install pydot graphviz

In [ ]:
from tensorflow.keras.utils import model_to_dot, plot_model
from IPython.display import SVG

In [ ]:
# from keras.utils import plot_model
plot_model(
    model, show_shapes=True, show_layer_names=True,
)

### Creating model, using existing one

In [ ]:
es = EarlyStopping(
    monitor="val_loss",
    verbose=1,
    mode="min",
    patience=patience,
    restore_best_weights=True,
)
# compile the model (should be done *after* setting layers to non-trainable)

# We need to recompile the model for these modifications to take effect
es.set_model(model)
loss = "binary_crossentropy" if OUTPUT_DIM == 2 else "categorical_crossentropy"
# optimizer = Lookahead(RectifiedAdam(), sync_period=6, slow_step_size=0.5)

model.compile(optimizer=optimizer, loss=loss)

### Fit new layers

In [ ]:
truth_test = []
for (_, _), labels in dataset_test:
    truth_test.append(np.argmax(labels, axis=1))
truth_test = np.hstack(truth_test)
truth_test

truth_valid = []
for i, ((_, _), labels) in enumerate(dataset_valid):
    truth_valid.append(np.argmax(labels, axis=1))
#     if i >= steps_per_epoch_val - 1:
#         break
truth_valid = np.hstack(truth_valid)
truth_valid.shape

### Unfreeze and fit more/all layers

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit(
    dataset_train,
    callbacks=[es],
    epochs=epochs,
    # steps_per_epoch=steps_per_epoch,
    validation_data=dataset_valid,
    # validation_steps=steps_per_epoch_val,
)

In [ ]:
plot_metric(history, "loss")

## Evaluation

In [ ]:
preds_valid = model.predict(dataset_valid)  # , steps=steps_per_epoch_val)
preds_valid.shape
if OUTPUT_DIM > 2:
    print(
        f"Accuracy valid: {accuracy_score(truth_valid, np.argmax(preds_valid, axis=1))}"
    )
if OUTPUT_DIM == 2:
    print(f"ROC AUC valid: {roc_auc_score(truth_valid, preds_valid[:, 1])}")
preds_test = model.predict(dataset_test)
preds_test.shape
if OUTPUT_DIM > 2:
    print(f"Accuracy test: {accuracy_score(truth_test, np.argmax(preds_test, axis=1))}")
if OUTPUT_DIM == 2:
    print(f"ROC AUC test: {roc_auc_score(truth_test, preds_test[:, 1])}")

In [ ]:
# Open payments 13min, 0.9338321957865205 (batch 256)
# cat-in-the-dat-ii 0.7643929251726093 1h10 (batch 256)
# RL ROC AUC test: 0.9430279072306678 mobile net patience 20
# BNP cardif (image size : 96) ROC AUC test: 0.7168011573597879 1h16
# give-me-some-credit ROC AUC test: 0.8591709381962962 2h4

In [ ]:
# Open payments 1h10
# ROC AUC valid: 0.9432323040315754
# ROC AUC test: 0.9345809216081824

In [ ]:
# EfficientNetB0
# EfficientNetB2 2h => ROC AUC valid: 0.9357130861335854 ROC AUC test: 0.9330937133279599

In [ ]:
# "santander-customer-satisfaction"
# 0.8164865598696714 => target size 96, whole re train
# 0.8141824599511267 => target size 224, whole re train
# ROC AUC valid: 0.8268400760249797 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout, earlystoping eevry time

# give me some credit
# 0.8462134942186483 => target size 160, whole train (batch 128, 2layers 1024, 128)

# ROC AUC valid: 0.8348917439829162
# RL
# ROC AUC valid: 0.892118469133795 => 160, whole, batch 64, 1layer 1024
# ROC AUC valid: 0.9233534348199217 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout
# ROC AUC valid: 0.9465346534653467 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout, earlystoping eevry time

# Open payment
# ROC AUC valid: 0.9186458210299415 => 96 =>160, whole, batch 64, 1layer 1024
# ROC AUC valid: 0.8778868370932499 => 96 => 96, whole, batch 64, 1layer 1024
# ROC AUC valid: 0.8917381493730192 => 96 => 224, whole, batch 64, 1layer 1024
# ROC AUC valid: 0.9090374872044725 => 96 => 96, whole, batch 64, 1layer 1024 -> 512 -> 128
# ROC AUC valid: 0.886417393797122 => 96 => 160, whole, batch 64, 1layer 1024 -> 512 -> 128
# ROC AUC valid: 0.9045819676568436 => 96 => 96, whole, batch 64, 1layer 1024 -> 128, no dropout
# ROC AUC valid: 0.9045819676568436 => 96 => 96, whole, batch 64, 1layer 1024 -> 128, no dropout
# ROC AUC valid: 0.9406275221953189 => 96 => 96, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout
# ROC AUC valid: 0.9399110034154216 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout, earlystoping eevry time
# ROC AUC valid: 0.8811034128677376 => 96 => 96, whole, batch 128, 1layer 1024 -> 512 -> 128, no dropout, earlystoping eevry time

# Bank marketing
# ROC AUC valid: 0.7970734141661526 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout

# Albert
# ROC AUC valid: 0.7500980687987842 => 288 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout

# bnp-cardif
# ROC AUC valid: 0.7206667869818926 => 384 => 96, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout


In [ ]:
# "santander-customer-satisfaction"
# 0.833417731838137 => target size 96, whole train
# 0.8170226029745679 => target size 224, whole train
# ROC AUC test: 0.8348549041045967 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout, earlystoping eevry time

# give me some credit
# 0.8453497574694486 => target size 160, whole train (batch 128, 2layers 1024, 128)
# ROC AUC test: 0.8447058873195916

# RL
# ROC AUC test: 0.9051288159651395 => 160, whole, batch 64, 1layer 1024
# ROC AUC test: 0.9128674518211912 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout
# ROC AUC test: 0.9456874816987527 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout, earlystoping eevry time
# ROC AUC test: 0.9399110034154216 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout, earlystoping eevry time

# Open payment
# ROC AUC test: 0.916907759155486 => 96 =>160, whole, batch 64, 1layer 1024
# ROC AUC test: 0.8931101859362467 => 96 => 96, whole, batch 64, 1layer 1024
# ROC AUC test: 0.8938387451368033 => 96 => 224, whole, batch 64, 1layer 1024
# ROC AUC test: 0.9067445823812874 => 96 => 96, whole, batch 64, 1layer 1024 -> 512 -> 128
# ROC AUC test: 0.8954549557710788 => 96 => 160, whole, batch 64, 1layer 1024 -> 512 -> 128
# ROC AUC test: 0.9062895529860363 => 96 => 96, whole, batch 64, 1layer 1024 -> 128, no dropout
# ROC AUC test: 0.9403517762951931 => 96 => 96, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout

# Bank marketing
# ROC AUC valid: 0.7959000291791145 => 160 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout

# Albert
# ROC AUC test: 0.7487660412524685 => 288 => 160, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout

# bnp-cardif
# ROC AUC test: 0.725934546476426 => 384 => 96, whole, batch 32, 1layer 1024 -> 512 -> 128, no dropout
